In [1]:
# @title Warnings Filter
import warnings
warnings.filterwarnings("ignore")

In [2]:
# @title Runtime selection
import torch
runtime= torch.device("cuda" if torch.cuda.is_available() else "cpu")
runtime

device(type='cpu')

In [3]:
# @title Google Drive
from google.colab import drive
drive.mount('/content/drive')
trainpath = '/content/drive/My Drive/train'
testpath = '/content/drive/My Drive/test'

Mounted at /content/drive


In [4]:
# @title Image Preprocessing function
from torchvision import transforms
trans=transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor(),transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

In [5]:
# @title Dataset loading Training Set and Test Set
from torchvision import datasets
trainset=datasets.ImageFolder(trainpath,trans)
testset=datasets.ImageFolder(testpath,trans)
trainset.classes

['0', '1']

In [6]:
# @title Initializing the Batch Size
from torch.utils.data import DataLoader
train=DataLoader(trainset,batch_size=32,shuffle=True,num_workers=4)
test=DataLoader(testset,batch_size=32,shuffle=True,num_workers=4)

In [7]:
# @title Importing the pretrained ResNet18 model
from torchvision import models
model=models.resnet18(pretrained=True)
model.fc

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 136MB/s]


Linear(in_features=512, out_features=1000, bias=True)

In [8]:
# @title Modifiying the ResNet18 to Binary Classifying Model
import torch.nn
count_inp_feat=model.fc.in_features
model.fc=torch.nn.Sequential(torch.nn.Dropout(p=0.5),torch.nn.Linear(count_inp_feat,2))
model.fc

Sequential(
  (0): Dropout(p=0.5, inplace=False)
  (1): Linear(in_features=512, out_features=2, bias=True)
)

In [9]:
# @title Loss and Optimizer
import torch.optim
import torch.optim.lr_scheduler as scheduler
loss=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.0001)
sched=scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [ ]:
# @title Training
import torch
epochs=5
model.to(runtime)
for ep in range(epochs):
  model.train()
  loss_overall,cor,tot=0,0,0
  for inp,lab in train:
    inp,lab=inp.to(runtime),lab.to(runtime)
    optimizer.zero_grad()
    out=model(inp)
    loss_current=loss(out,lab)
    loss_current.backward()
    optimizer.step()
    loss_overall+=loss_current.item()
    x,pred=torch.max(out.data,1)
    cor+=(pred==lab).sum().item()
    tot+=lab.size(0)
  print("Epoch:",ep+1,"Loss:",'%.2f'%(loss_overall/len(train)),"Accuracy:",'%.2f'%(cor/tot*100))
  sched.step()

Epoch: 1 Loss: 0.42 Accuracy: 81.12
Epoch: 2 Loss: 0.27 Accuracy: 88.88
Epoch: 3 Loss: 0.20 Accuracy: 91.88
Epoch: 4 Loss: 0.15 Accuracy: 93.85
Epoch: 5 Loss: 0.11 Accuracy: 95.56


In [23]:
# @title Testing and Evaluation
model.eval()
for inp,lab in train:
  inp,lab=inp.to(runtime),lab.to(runtime)
  out=model(inp)
  x,pred=torch.max(out.data,1)
  cor+=(pred==lab).sum().item()
  tot+=lab.size(0)
print("Accuracy:",'%.2f'%(cor/tot*100))

Accuracy: 99.83
